<center>
    <img src="https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/Logos/organization_logo/organization_logo.png" width="300" alt="cognitiveclass.ai logo"  />
</center>

# Assignment: Notebook for Peer Assignment

Estimated time needed: 45 minutes


# Assignment Scenario

Congratulations! You have just been hired by a US Venture Capital firm as a data analyst.

The company is considering foreign grain markets to help meet its supply chain requirements for its recent investments in the microbrewery and microdistillery industry, which is involved with the production and distribution of craft beers and spirits.

Your first task is to provide a high level analysis of crop production in Canada. Your stakeholders want to understand the current and historical performance of certain crop types in terms of supply and price volatility. For now they are mainly interested in a macro-view of Canada's crop farming industry, and how it relates to the relative value of the Canadian and US dollars.


# Introduction

Using this R notebook you will:

1.  Understand four datasets 
2.  Load the datasets into four separate tables in a Db2 database
3.  Execute SQL queries unsing the RODBC R package to answer assignment questions 

You have already encountered two of these datasets in the previous practice lab. You will be able to reuse much of the work you did there to prepare your database tables for executing SQL queries.


# Understand the datasets

To complete the assignment problems in this notebook you will be using subsetted snapshots of two datasets from Statistics Canada, and one from the Bank of Canada. The links to the prepared datasets are provided in the next section; the interested student can explore the landing pages for the source datasets as follows:

1.  <a href="https://www150.statcan.gc.ca/t1/tbl1/en/tv.action?pid=3210035901">Canadian Principal Crops (Data & Metadata)</a>
2.  <a href="https://www150.statcan.gc.ca/t1/tbl1/en/tv.action?pid=3210007701">Farm product prices (Data & Metadata)</a>
3.  <a href="https://www.bankofcanada.ca/rates/exchange/daily-exchange-rates">Bank of Canada daily average exchange rates</a>

### 1. Canadian Principal Crops Data *

This dataset contains agricultural production measures for the principle crops grown in Canada, including a breakdown by province and teritory, for each year from 1908 to 2020.

For this assignment you will use a preprocessed snapshot of this dataset (see below).

A detailed description of this dataset can be obtained from the StatsCan Data Portal at:
[https://www150.statcan.gc.ca/t1/tbl1/en/tv.action?pid=3210035901](https://www150.statcan.gc.ca/t1/tbl1/en/tv.action?pid=3210035901&cm_mmc=Email_Newsletter-_-Developer_Ed%2BTech-_-WW_WW-_-SkillsNetwork-Courses-IBM-RP0203EN-SkillsNetwork-23863830&cm_mmca1=000026UJ&cm_mmca2=10006555&cm_mmca3=M12345678&cvosrc=email.Newsletter.M12345678&cvo_campaign=000026UJ&cm_mmc=Email_Newsletter-_-Developer_Ed%2BTech-_-WW_WW-_-SkillsNetwork-Courses-IBM-RP0203EN-SkillsNetwork-23863830&cm_mmca1=000026UJ&cm_mmca2=10006555&cm_mmca3=M12345678&cvosrc=email.Newsletter.M12345678&cvo_campaign=000026UJ)  
Detailed information is included in the metadata file and as header text in the data file, which can be downloaded - look for the 'download options' link.  

### 2. Farm product prices

This dataset contains monthly average farm product prices for Canadian crops and livestock by province and teritory, from 1980 to 2020 (or 'last year', whichever is greatest).

For this assignment you will use a preprocessed snapshot of this dataset (see below).

A description of this dataset can be obtained from the StatsCan Data Portal at:
[https://www150.statcan.gc.ca/t1/tbl1/en/tv.action?pid=3210007701](https://www150.statcan.gc.ca/t1/tbl1/en/tv.action?pid=3210007701&cm_mmc=Email_Newsletter-_-Developer_Ed%2BTech-_-WW_WW-_-SkillsNetwork-Courses-IBM-RP0203EN-SkillsNetwork-23863830&cm_mmca1=000026UJ&cm_mmca2=10006555&cm_mmca3=M12345678&cvosrc=email.Newsletter.M12345678&cvo_campaign=000026UJ) 
The information is included in the metadata file, which can be downloaded - look for the 'download options' link.  

### 3. Bank of Canada daily average exchange rates *

This dataset contains the daily average exchange rates for multiple foreign currencies. Exchange rates are expressed as 1 unit of the foreign currency converted into Canadian dollars. It includes only the latest four years of data, and the rates are published once each business day by 16:30 ET.

For this assignment you will use a snapshot of this dataset with only the USD-CAD exchange rates included (see next section). We have also prepared a monthly averaged version which you will be using below.

A brief description of this dataset and the original dataset can be obtained from the Bank of Canada Data Portal at:
[https://www.bankofcanada.ca/rates/exchange/daily-exchange-rates/](https://www.bankofcanada.ca/rates/exchange/daily-exchange-rates?cm_mmc=Email_Newsletter-_-Developer_Ed%2BTech-_-WW_WW-_-SkillsNetwork-Courses-IBM-RP0203EN-SkillsNetwork-23863830&cm_mmca1=000026UJ&cm_mmca2=10006555&cm_mmca3=M12345678&cvosrc=email.Newsletter.M12345678&cvo_campaign=000026UJ)

( * these datasets are the same as the ones you used in the practice lab)


### Dataset URLs

1.  Annual Crop Data: <https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-RP0203EN-SkillsNetwork/labs/Final%20Project/Annual_Crop_Data.csv> 

2.  Farm product prices: <https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-RP0203EN-SkillsNetwork/labs/Final%20Project/Monthly_Farm_Prices.csv>

3.  Daily FX Data: <https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-RP0203EN-SkillsNetwork/labs/Final%20Project/Daily_FX.csv>

4.  Monthly FX Data: <https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-RP0203EN-SkillsNetwork/labs/Final%20Project/Monthly_FX.csv>

<span style="color:red">**IMPORTANT:**</span> You will be loading these datasets directly into R data frames from these URLs instead of from the StatsCan and Bank of Canada portals. The versions provided at these URLs are simplified and subsetted versions of the original datasets.


#### Now let's load these datasets into four separate Db2 tables.

Let's first load the RODBC package:


In [1]:
import numpy as np
import pandas as pd
import datetime

# Establish database connection
import sqlite3 as sq3
import ibm_db
import ibm_db_dbi
%load_ext sql

pd.set_option('display.max_columns',None)
#pd.set_option('display.max_rows',None)
pd.set_option('display.width', 1000)
pd.set_option('display.float_format','{:.2f}'.format)

## Problem 1

#### Create tables

Establish a connection to the Db2 database, and create the following four tables using the RODBC package in R. 
Use the separate cells provided below to create each of your tables.

1.  **CROP_DATA**
2.  **FARM_PRICES**
3.  **DAILY_FX**
4.  **MONTHLY_FX**  

The previous practice lab will help you accomplish this.


### Solution 1


### IBM DB

In [ ]:
#Replace the placeholder values with your actual Db2 hostname, username, and password:
dsn_hostname = "dashdb-txn-sbox-yp-dal09-04.services.dal.bluemix.net" # e.g.: "dashdb-txn-sbox-yp-dal09-04.services.dal.bluemix.net"
dsn_uid = "glm90595"        # e.g. "abc12345"
dsn_pwd = "m3rrl046qnqcx^35"      # e.g. "7dBZ3wWt9XN6$o0J"

dsn_driver = "{IBM DB2 ODBC DRIVER}"
dsn_database = "BLUDB"            # e.g. "BLUDB"
dsn_port = "50000"                # e.g. "50000" 
dsn_protocol = "TCPIP"            # i.e. "TCPIP"

In [ ]:
#DO NOT MODIFY THIS CELL. Just RUN it with Shift + Enter
#Create the dsn connection string
dsn = (
    "DRIVER={0};"
    "DATABASE={1};"
    "HOSTNAME={2};"
    "PORT={3};"
    "PROTOCOL={4};"
    "UID={5};"
    "PWD={6};").format(dsn_driver, dsn_database, dsn_hostname, dsn_port, dsn_protocol, dsn_uid, dsn_pwd)

#print the connection string to check correct values are specified
print(dsn)

In [ ]:
#DO NOT MODIFY THIS CELL. Just RUN it with Shift + Enter
#Create database connection

try:
    conn = ibm_db.connect(dsn, "", "")
    print ("Connected to database: ", dsn_database, "as user: ", dsn_uid, "on host: ", dsn_hostname)

except:
    print ("Unable to connect: ", ibm_db.conn_errormsg() )


In [ ]:
#Retrieve Metadata for the Database Server
server = ibm_db.server_info(conn)

print ("DBMS_NAME: ", server.DBMS_NAME)
print ("DBMS_VER:  ", server.DBMS_VER)
print ("DB_NAME:   ", server.DB_NAME)

In [ ]:
#Retrieve Metadata for the Database Client / Driver
client = ibm_db.client_info(conn)

print ("DRIVER_NAME:          ", client.DRIVER_NAME) 
print ("DRIVER_VER:           ", client.DRIVER_VER)
print ("DATA_SOURCE_NAME:     ", client.DATA_SOURCE_NAME)
print ("DRIVER_ODBC_VER:      ", client.DRIVER_ODBC_VER)
print ("ODBC_VER:             ", client.ODBC_VER)
print ("ODBC_SQL_CONFORMANCE: ", client.ODBC_SQL_CONFORMANCE)
print ("APPL_CODEPAGE:        ", client.APPL_CODEPAGE)
print ("CONN_CODEPAGE:        ", client.CONN_CODEPAGE)

In [ ]:
ibm_db.close(conn)

### SQLite

### Test Connection

In [ ]:
con = sq3.connect("sqlite_data.db")

In [ ]:
con.execute("Select * FROM sqlite_master").fetchall() #Check no data

In [ ]:
con.close()

### Create Tables

In [ ]:
# CROP_DATA:
df1 = pd.read_csv("Annual_Crop_Data.csv", parse_dates=["YEAR"])
df1.head()

In [ ]:
CROP_DATA = df1.copy()

In [ ]:
# FARM_PRICES:
df2 = pd.read_csv("Monthly_Farm_Prices.csv", parse_dates=["date"])
df2.head()

In [ ]:
FARM_PRICES = df2.copy()

In [ ]:
# DAILY_FX:
df3 = pd.read_csv("Daily_FX.csv", parse_dates=["date"])
df3.head()

In [ ]:
DAILY_FX = df3.copy()

In [ ]:
# MONTHLY_FX:
df4 = pd.read_csv("Monthly_FX.csv", parse_dates=["date"])
df4.head()

In [ ]:
MONTHLY_FX = df4.copy()

### Upload to Database

In [ ]:
con = sq3.connect("sqlite_data.db")

In [ ]:
con

In [ ]:
CROP_DATA.to_sql("Cropdata", con, index=False)

In [ ]:
FARM_PRICES.to_sql("Farmprices", con, index=False)

In [ ]:
DAILY_FX.to_sql("Dailyfx", con, index=False)

In [ ]:
MONTHLY_FX.to_sql("Monthlyfx", con, index=False)

In [ ]:
con.execute("Select * FROM sqlite_master").fetchall()

In [ ]:
pd.read_sql_query("Select * FROM sqlite_master", con) #Check all tables inside database

In [ ]:
con.close()

## Problem 2

#### Read Datasets and Load Tables

Read the datasets into R dataframes using the urls provided above. Then load your tables.


### Solution 2


In [2]:
db = sq3.connect("sqlite_data.db") #Create connection

In [3]:
cursor = db.cursor()

In [4]:
pd.read_sql_query('SELECT * FROM sqlite_master', db) #Check all tables

,type,name,tbl_name,rootpage,sql
0,table,Cropdata,Cropdata,2,"CREATE TABLE ""Cropdata"" (\n""index"" INTEGER,\n ..."
1,table,Farmprices,Farmprices,13,"CREATE TABLE ""Farmprices"" (\n""index"" INTEGER,\..."
2,table,Dailyfx,Dailyfx,50,"CREATE TABLE ""Dailyfx"" (\n""index"" INTEGER,\n ..."
3,table,Monthlyfx,Monthlyfx,61,"CREATE TABLE ""Monthlyfx"" (\n""index"" INTEGER,\n..."


## Now execute SQL queries using the RODBC R package to solve the assignment problems.

## Problem 3

#### How many records are in the farm prices dataset?


### Solution 3


In [18]:
farmprice = pd.read_sql_query('SELECT * FROM Farmprices', db , index_col="index", parse_dates='date')

In [19]:
farmprice

,date,cropType,GEO,pricePerMT
index,,,,
0,1985-01-01,Barley,Alberta,127.39
1,1985-01-01,Barley,Saskatchewan,121.38
2,1985-01-01,Canola,Alberta,342.00
3,1985-01-01,Canola,Saskatchewan,339.82
4,1985-01-01,Rye,Alberta,100.77
...,...,...,...,...
2673,2020-12-01,Canola,Saskatchewan,507.33
2674,2020-12-01,Rye,Alberta,202.41
2675,2020-12-01,Rye,Saskatchewan,192.90


In [20]:
pd.read_sql_query('SELECT COUNT(*) FROM Farmprices', db)

,COUNT(*)
0,2678


In [9]:
farmprice.shape

(2678, 4)

## Problem 4

#### Which geographies are included in the farm prices dataset?


### Solution 4

In [22]:
pd.read_sql_query('SELECT DISTINCT GEO FROM Farmprices', db)

,GEO
0,Alberta
1,Saskatchewan


In [21]:
farmprice.GEO.unique()

array(['Alberta', 'Saskatchewan'], dtype=object)

## Problem 5

#### How many hectares of Rye were harvested in Canada in 1968?


In [ ]:
cropdata = pd.read_sql_query('SELECT * FROM Cropdata',db , parse_dates='YEAR')

In [ ]:
cropdata

### Solution 5

In [ ]:
cd = cropdata[(cropdata["YEAR"] == "1968-12-31") & (cropdata["cropType"] == "Rye")]
cd

In [ ]:
cd.harvestedArea.sum()

## Problem 6

#### Query and display the first 6 rows of the farm prices table for Rye.


### Solution 6


In [ ]:
ryeprice = farmprice[farmprice["cropType"] == "Rye"]

In [ ]:
ryeprice.head(6)

## Problem 7

#### Which provinces grew Barley?


### Solution 7


In [ ]:
cropdata.head()

In [ ]:
barley_provinces = cropdata[cropdata["cropType"] == "Barley"]

In [ ]:
barley_provinces.GEO.unique()

## Problem 8

#### Find the first and last dates for the farm prices data.


### Solution 8


In [ ]:
farmprice['date'].min()

In [ ]:
farmprice['date'].max()

## Problem 9

#### Which crops have ever reached a farm price greater than or equal to $350 per metric tonne?


### Solution 9


In [ ]:
croptype = farmprice[farmprice["pricePerMT"] >= 350.00]
croptype

In [ ]:
croptype.cropType.unique()

## Problem 10

#### Rank the crop types harvested in Saskatchewan in the year 2000 by their average yield. Which crop performed best?


### Solution 10


In [ ]:
cropdata

In [ ]:
cropdata["year"] = cropdata["YEAR"].dt.year

In [ ]:
cropdata.head()

In [ ]:
year2000 = cropdata[cropdata["year"] == 2000]

In [ ]:
year2000

In [ ]:
year2000[year2000["GEO"] == "Saskatchewan"].sort_values(by='avgYield')

## Problem 11

#### Rank the crops and geographies by their average yield (KG per hectare) since the year 2000. Which crop and province had the highest average yield since the year 2000?


### Solution 11


In [ ]:
rank = cropdata[cropdata["year"] >= 2000]
rank

In [ ]:
rank.groupby(['cropType','GEO'])['avgYield'].sum().sort_values()

## Problem 12

#### Use a subquery to determine how much wheat was harvested in Canada in the most recent year of the data.


### Solution 12


In [ ]:
year2020 = cropdata[cropdata["year"] == 2020]

In [ ]:
year2020

In [ ]:
harvest = year2020[year2020["cropType"] == "Wheat"]
harvest

In [ ]:
harvest.harvestedArea.sum()

## Problem 13

#### Use an implicit inner join to calculate the monthly price per metric tonne of Canola grown in Saskatchewan in both Canadian and US dollars. Display the most recent 6 months of the data.


### Solution 13


In [ ]:
dailyfx = pd.read_sql_query('SELECT * FROM Dailyfx', db , parse_dates='date')

In [ ]:
dailyfx.head()

In [ ]:
monthlyfx = pd.read_sql_query('SELECT * FROM Monthlyfx', db , parse_dates='date')

In [ ]:
monthlyfx.head()

In [ ]:
canola = farmprice[farmprice["cropType"] == "Canola"]

In [ ]:
canola

In [ ]:
canolasas = canola[canola["GEO"] == "Saskatchewan"]

In [ ]:
canolasas

In [ ]:
canolasas2020 = canolasas[canolasas["year"] == 2020]

In [ ]:
canolasas2020

In [ ]:
df = pd.merge(left=canolasas2020, right=monthlyfx, on="date", how="inner")

In [ ]:
df

In [ ]:
df["CAD"] = df["pricePerMT"] * 1.00

In [ ]:
df["USD"] = df["CAD"] / df["FXUSDCAD"]

In [ ]:
df.tail(6)

In [24]:
db.close() ### Close connection

## Author(s)

<h4> Jeff Grossman </h4>

## Contributor(s)

<h4> Rav Ahuja </h4>

## Change log

| Date       | Version | Changed by    | Change Description                                                                                         |
| ---------- | ------- | ------------- | ---------------------------------------------------------------------------------------------------------- |
| 2021-04-01 | 0.7     | Jeff Grossman | Split Problem 1 solution cell into multiple cells, fixed minor bugs                                        |
| 2021-03-12 | 0.6     | Jeff Grossman | Cleaned up content for production                                                                          |
| 2021-03-11 | 0.5     | Jeff Grossman | Moved more advanced problems to optional honours module                                                    |
| 2021-03-10 | 0.4     | Jeff Grossman | Added introductory and intermediate level problems and removed some advanced problems                      |
| 2021-03-04 | 0.3     | Jeff Grossman | Moved some problems to a new practice lab as prep for this assignment                                      |
| 2021-03-04 | 0.2     | Jeff Grossman | Sorted problems roughly by level of difficulty and relegated more advanced ones to ungraded bonus problems |
| 2021-02-20 | 0.1     | Jeff Grossman | Started content creation                                                                                   |

## <h3 align="center"> © IBM Corporation 2021. All rights reserved. <h3/>
